In [ ]:
# Data Source: Inside Airbnb
# Snapshot: June 2025
# File: listings.csv (summary, 18 columns)
# Goal: city-level preprocessing + light EDA
# Note: Do NOT drop columns, row-level cleaning only

In [ ]:
# 구글 드라이브 마운트
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
#데이터로드 및 데이터크기 확인
import pandas as pd

path = '/content/drive/MyDrive/NewYork_listings.csv'
df = pd.read_csv(path, low_memory=False)

df.shape

(36322, 18)

In [ ]:
df.head()

,id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365,number_of_reviews_ltm,license
0,2539,Superfast Wi-Fi. Clean & quiet home by the park,2787,John,Brooklyn,Kensington,40.64529,-73.97238,Private room,285.0,30,9,2018-10-19,0.08,6,347,0,NaN
1,2595,Skylit Midtown Manhattan Spacious Studio,2845,Jennifer,Manhattan,Midtown,40.75356,-73.98559,Entire home/apt,240.0,30,49,2022-06-21,0.26,3,312,0,NaN
2,5136,Spacious Family Friendly Duplex w/ Patio + Yard,7378,Rebecca,Brooklyn,Sunset Park,40.66265,-73.99454,Entire home/apt,215.0,30,4,2023-08-20,0.03,1,80,0,NaN
3,6848,Only 2 stops to Manhattan studio,15991,Allen,Brooklyn,Williamsburg,40.70935,-73.95342,Entire home/apt,97.0,30,196,2025-06-09,1.00,1,219,4,NaN
4,6872,Uptown Sanctuary w/ Private Bath (Month to Month),16104,Kahshanna,Manhattan,East Harlem,40.80107,-73.94255,Private room,NaN,30,1,2022-06-05,0.03,2,2,0,NaN


In [ ]:
df.info()
df.shape

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 36322 entries, 0 to 36321
Data columns (total 18 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   id                              36322 non-null  int64  
 1   name                            36320 non-null  object 
 2   host_id                         36322 non-null  int64  
 3   host_name                       36307 non-null  object 
 4   neighbourhood_group             36322 non-null  object 
 5   neighbourhood                   36322 non-null  object 
 6   latitude                        36322 non-null  float64
 7   longitude                       36322 non-null  float64
 8   room_type                       36322 non-null  object 
 9   price                           21459 non-null  float64
 10  minimum_nights                  36322 non-null  int64  
 11  number_of_reviews               36322 non-null  int64  
 12  last_review                     

(36322, 18)

In [ ]:
# 결측치 확인
# 결측치 비율 계산
missing_ratio = df.isna().mean().sort_values(ascending=False)

missing_ratio

,0
license,0.853285
price,0.409201
reviews_per_month,0.307004
last_review,0.307004
host_name,0.000413
name,0.000055
id,0.000000
neighbourhood,0.000000
neighbourhood_group,0.000000
host_id,0.000000


In [ ]:
#결측치는 무작위 아니고 숙소 운영 상태를 반영한 구조적 결측으로 판단해 규칙기반으로 처리
# price가 없는 숙소는 가격 분석 불가 → 제거
df = df.dropna(subset=["price"])

# 리뷰가 없는 숙소는 리뷰 0으로 해석
df["reviews_per_month"] = df["reviews_per_month"].fillna(0)

df.shape

(21459, 18)

In [ ]:

# 각 컬럼의 데이터 타입 확인
df.dtypes

,0
id,int64
name,object
host_id,int64
host_name,object
neighbourhood_group,object
neighbourhood,object
latitude,float64
longitude,float64
room_type,object
price,float64


In [ ]:
#중복확인
df.duplicated().sum()
df["id"].duplicated().sum()

np.int64(0)

In [ ]:
#기본통계정보 확인 (이상치나 값 패턴)
df.describe()

,id,host_id,latitude,longitude,price,minimum_nights,number_of_reviews,reviews_per_month,calculated_host_listings_count,availability_365,number_of_reviews_ltm
count,2.145900e+04,2.145900e+04,21459.000000,21459.000000,21459.000000,21459.000000,21459.000000,21459.000000,21459.000000,21459.000000,21459.000000
mean,6.149346e+17,2.091221e+08,40.727775,-73.944380,521.528356,27.469873,35.545319,0.799522,84.881076,245.644205,6.269770
std,5.301014e+17,2.018163e+08,0.059228,0.060472,3501.384801,33.742030,80.932657,1.946193,234.721884,106.924975,25.106628
min,2.539000e+03,1.678000e+03,40.500366,-74.251907,8.000000,1.000000,0.000000,0.000000,1.000000,0.000000,0.000000
25%,4.189217e+07,2.512704e+07,40.685615,-73.983615,89.000000,30.000000,0.000000,0.000000,1.000000,167.000000,0.000000
50%,7.421051e+17,1.213795e+08,40.728090,-73.954070,154.000000,30.000000,5.000000,0.200000,3.000000,270.000000,0.000000
75%,1.094826e+18,4.073020e+08,40.763410,-73.922245,270.000000,30.000000,37.000000,0.920000,20.000000,342.000000,3.000000
max,1.444534e+18,7.005911e+08,40.905919,-73.711822,50184.000000,730.000000,3277.000000,121.970000,1054.000000,365.000000,1786.000000


In [ ]:
#price 상위 몇 개만 확인
df.nlargest(10, "price")[["id","price","minimum_nights","room_type","neighbourhood"]]

,id,price,minimum_nights,room_type,neighbourhood
29749,1067326290090101851,50184.0,1,Hotel room,Hell's Kitchen
35464,1394977539353506309,50052.0,1,Hotel room,Lower East Side
35529,1386264308699480110,50052.0,1,Hotel room,Nolita
35531,1386264479344931911,50052.0,1,Hotel room,Lower East Side
25212,836129981914196311,50046.0,1,Hotel room,Hell's Kitchen
27220,920103826004223083,50046.0,1,Hotel room,Hell's Kitchen
29466,1042054162835980391,50046.0,1,Hotel room,Hell's Kitchen
29751,1067335526410187895,50046.0,1,Hotel room,Hell's Kitchen
23864,772490456980779766,50029.0,1,Hotel room,Midtown
29170,1035513737232864814,50002.0,1,Hotel room,Murray Hill


In [ ]:
#minimum_night 상위 몇 개 확인
df.nlargest(10, "minimum_nights")[["id","minimum_nights","price"]]

,id,minimum_nights,price
13150,34974475,730,100.0
13421,36454717,730,71.0
13422,36455321,730,55.0
13423,36455649,730,71.0
13610,35875858,730,71.0
13617,35898731,730,55.0
34875,1356624484375250156,730,17.0
26905,910554926647877661,729,68.0
27094,913323222250021583,729,77.0
27095,913341921156977472,729,77.0


1. price : 상단꼬리(극단값) 매우김  

-  “NYC 가격 중앙값 154, 상위 25%는 270 이상. 극단값(5만) 존재 → 추후 이상치 기준 논의 필요”  

2. minimum_nights : 중앙값 30  

- 이건 NYC 시장(또는 규정)의 특징일 가능성이 커 보임.“NYC는 minimum_nights가 30에 강하게 몰림(규제/장기임대 성격 가능)”  

3. availability_365 — 중앙값 270, 평균 245  

-  “연중 많이 열려 있는 숙소가 많다”는 신호.(수요가 낮다/높다 단정은 아직 ❌)   

4. 리뷰 지표 — 0이 많고, 일부가 매우 큼  

-  전형적으로 “리뷰 없는 숙소 많음 + 소수 숙소가 매우 활발” 패턴.
- “리뷰 지표 0 비중 큼(특히 하위 25%는 0), 극단적으로 활발한 숙소 존재”  

5. calculated_host_listings_count — max 1054 (초대형 호스트 존재)  

-  “대부분은 소규모 호스트, 일부는 기업형 호스트” 패턴.
- “호스트 규모 분포가 매우 넓음(기업형 호스트 존재 가능)”  


In [ ]:
#price_usd 컬럼 추가하기
# NYC: price는 이미 USD (Inside Airbnb NYC)
df["price_usd"] = (
    df["price"].astype(str)
      .str.replace(r"[$,]", "", regex=True)
      .astype(float)
)


In [ ]:
#도시 필터 적용을 위해 city 컬럼 추가
df["city"] = "NYC"

In [ ]:
# 클린 파일 저장 (price 없는 숙소는 제거했고 리뷰없는 숙소는 0으로 해석함.)
import os

# 저장할 폴더 경로
dir_path = "/content/drive/MyDrive/airbnb/clean"

# 폴더가 없으면 생성
os.makedirs(dir_path, exist_ok=True)

save_path = "/content/drive/MyDrive/airbnb/clean/nyc_listings_jun2025_summary_clean.csv"
df.to_csv(save_path, index=False)

In [ ]:
# 파일 존재 확인
os.path.exists(save_path)

True

In [ ]:
pd.read_csv(save_path).shape

(21459, 20)

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 21459 entries, 0 to 36321
Data columns (total 20 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   id                              21459 non-null  int64  
 1   name                            21459 non-null  object 
 2   host_id                         21459 non-null  int64  
 3   host_name                       21454 non-null  object 
 4   neighbourhood_group             21459 non-null  object 
 5   neighbourhood                   21459 non-null  object 
 6   latitude                        21459 non-null  float64
 7   longitude                       21459 non-null  float64
 8   room_type                       21459 non-null  object 
 9   price                           21459 non-null  float64
 10  minimum_nights                  21459 non-null  int64  
 11  number_of_reviews               21459 non-null  int64  
 12  last_review                     15126